# Progetto di Computer Science

## Davide Meloni, Alberto Raimondi
### Febbraio 2018

In [1]:
# importazione delle librerie
import pandas as pd
import numpy as np
import json
import operator
import re
import math
import random

In [2]:
#importo i CSV
df_movies=pd.read_csv("tmdb_5000_movies.csv")
df_credits=pd.read_csv("tmdb_5000_credits.csv")
df_credits["id"]=df_credits["movie_id"]
df=df_movies.merge(df_credits, on=['id','title'],how='left')

1 For each movie, compute the number of cast members

In [3]:
#conto il numero di attori in base all'id
df["no. cast members"]=df.cast.apply(lambda x: len(json.loads(x)))
df[["title","no. cast members"]].head()

,title,no. cast members
0,Avatar,83
1,Pirates of the Caribbean: At World's End,34
2,Spectre,83
3,The Dark Knight Rises,158
4,John Carter,27


2 How many movies do not have a homepage?

In [4]:
#conto il numero di film senza homepage in base all'id
df[df["homepage"].isnull()]['id'].count()

3091

3 For each year, how many movies do not have a homepage?

In [5]:
#creo la colonna "year" dai primi quattro caratteri dell'attributo "release_date" e conto il numero di film senza homepage per ogni anno in base all'id
df["year"]=df["release_date"].str[:4]
df[df["homepage"].isnull()].groupby("year").count()["id"]

year
1916      1
1925      1
1927      1
1929      2
1930      1
1932      1
1933      2
1934      1
1935      1
1936      2
1937      1
1938      2
1939      2
1940      4
1941      1
1942      1
1944      2
1945      4
1946      3
1947      3
1948      3
1949      2
1950      1
1951      3
1952      4
1953      4
1954      4
1955      2
1956      3
1957      2
       ... 
1987     29
1988     28
1989     28
1990     24
1991     29
1992     29
1993     44
1994     50
1995     64
1996     86
1997    102
1998    119
1999    145
2000    145
2001    159
2002    164
2003    131
2004    149
2005    149
2006    144
2007     92
2008     97
2009     85
2010     54
2011     43
2012     92
2013    127
2014    157
2015    110
2016     31
Name: id, Length: 89, dtype: int64

4 Extract the domain of each homepage.

In [6]:
# definisco una regex che vada a pulire i domini
def find_domain(string):
    if type(string) is str:
        final=[]
        temp=string.split(' ')#checking for movies with more than one homepages
        for i in temp:
            i=re.sub("(.*)\//", '', i)#removing everything before the double slash
            i=re.sub("www\.", '', i) #removing www.
            i=re.sub("\/.*", '', i) #removing the part after the slash
            #string=re.sub('\.com.*|\.net.*|\.co\.uk*|\.org.*', '', string)
            final.append(i)
        return ' '.join(final)
    else: 
        return float('NaN')

In [7]:
df["domain"]=df.homepage.apply(lambda x: find_domain(x))

In [8]:
#il film con due homepage viene trattato corettamente
for index,row in df.iterrows():
    if (' '  in str(row["domain"])):
        print (row['domain'])

cargoderfilm.ch cargothemovie.com


In [9]:
#controllo se i NaN son rimasti lo stesso numero
df["homepage"].isnull().sum()==(df["domain"].isnull()).sum()
#potrei rimuovere i nulli con 
#df['homepage'].dropna()
#df['domain'].dropna()

True

In [10]:
#creo la colonna "domain" facendo lo split della colonna "homepage" e prendendo la seconda rispetto al doppio slash, la prima rispetto allo slash e l'ultima divisione rispetto al punto 
#df["domain"]=df["homepage"].str.split(" ").str[0].str.split("//").str[1].str.split("/").str[0].str.split(".").str[-1]
df["domain"].dropna()

0                        avatarmovie.com
1                          disney.go.com
2                       sonypictures.com
3                 thedarkknightrises.com
4                      movies.disney.com
5                       sonypictures.com
6                          disney.go.com
7                             marvel.com
8             harrypotter.warnerbros.com
9       batmanvsupermandawnofjustice.com
10                          superman.com
11                               mgm.com
12                         disney.go.com
13                         disney.go.com
14                        manofsteel.com
16                            marvel.com
17                         disney.go.com
18                      sonypictures.com
19                         thehobbit.com
20               theamazingspiderman.com
21                 robinhoodthemovie.com
22                         thehobbit.com
23                goldencompassmovie.com
25                      titanicmovie.com
26              

5 Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary). There is no need to use SQL for this point.

In [11]:
#Creo tabella movie_id con i campi da non normalizzare
movie_id = df[['id', 'budget', 'domain', 'original_language', 'original_title', 'overview', 'popularity', 'release_date','year', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count' ]]
movie_id = movie_id.rename(index=str, columns={'id': 'movie_id'})
movie_id = movie_id.set_index(['movie_id'])
movie_id.tail(5)

,budget,domain,original_language,original_title,overview,popularity,release_date,year,revenue,runtime,status,tagline,title,vote_average,vote_count
movie_id,,,,,,,,,,,,,,,
9367,220000,NaN,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,1992,2040920,81.0,Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
72766,9000,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,2011,0,85.0,Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
231617,0,hallmarkchannel.com,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,2013,0,120.0,Released,NaN,"Signed, Sealed, Delivered",7.0,6
126186,0,shanghaicalling.com,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,2012,0,98.0,Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
25975,0,NaN,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,2005-08-05,2005,0,90.0,Released,NaN,My Date with Drew,6.3,16


In [12]:
#Collego movie_id con i relativi generi
genere_id = pd.DataFrame()
for i in range(len(df['genres'])):
    unrolled_data = pd.read_json(df['genres'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    genere_id = pd.concat([genere_id, unrolled_data])
genere_id = genere_id.rename(index=str, columns={'id': 'genere_id'})
genere = genere_id[['genere_id', 'name']]
genere_id = genere_id[['movie_id','genere_id']].set_index(['movie_id','genere_id'])
genere_id.tail()

Empty DataFrame
Columns: []
Index: [(231617, 35.0), (231617, 18.0), (231617, 10749.0), (231617, 10770.0), (25975, 99.0)]

In [13]:
# Creo la tablella di riferimento per i generi
genere = genere.set_index(['genere_id'])
genere = genere.drop_duplicates() 
genere.head()

,name
genere_id,
28.0,Action
12.0,Adventure
14.0,Fantasy
878.0,Science Fiction
80.0,Crime


In [14]:
# creo la tebella degli id delle varie keywords
keywords_ids = pd.DataFrame()
for i in range(len(df['keywords'])):
    unrolled_data = pd.read_json(df['keywords'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    keywords_ids = pd.concat([keywords_ids, unrolled_data])
keywords_ids = keywords_ids.rename(index=str, columns={'id': 'keywords_id'})
keywords_link = keywords_ids[['keywords_id', 'name']]
keywords_ids = keywords_ids[['movie_id', 'keywords_id']].set_index(['movie_id', 'keywords_id'])
keywords_ids.head(10)

Empty DataFrame
Columns: []
Index: [(19995, 1463.0), (19995, 2964.0), (19995, 3386.0), (19995, 3388.0), (19995, 3679.0), (19995, 3801.0), (19995, 9685.0), (19995, 9840.0), (19995, 9882.0), (19995, 9951.0)]

In [15]:
#Creo la tabella che indica le keywords per ogni id
keywords_link = keywords_link.set_index(['keywords_id'])
keywords_link = keywords_link.drop_duplicates()
keywords_link.head()

,name
keywords_id,
1463.0,culture clash
2964.0,future
3386.0,space war
3388.0,space colony
3679.0,society


In [16]:
#Creo la tebella degli id delle compagnie di produzione
production_id = pd.DataFrame()
for i in range(len(df['production_companies'])):
    unrolled_data = pd.read_json(df['production_companies'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    production_id = pd.concat([production_id, unrolled_data])
production_id = production_id.rename(index=str, columns={'id': 'company_id'})
production_link = production_id[['company_id', 'name']]
production_id = production_id[['movie_id', 'company_id']].set_index(['movie_id', 'company_id'])
production_id.head(10)

Empty DataFrame
Columns: []
Index: [(19995, 289.0), (19995, 306.0), (19995, 444.0), (19995, 574.0), (285, 2.0), (285, 130.0), (285, 19936.0), (206647, 5.0), (206647, 10761.0), (206647, 69434.0)]

In [17]:
#collego gli id ai nomi
#production_link = production_link.set_index(['company_id'])
production_link = production_link.drop_duplicates()
production_link.head()

,company_id,name
0.0,289.0,Ingenious Film Partners
1.0,306.0,Twentieth Century Fox Film Corporation
2.0,444.0,Dune Entertainment
3.0,574.0,Lightstorm Entertainment
0.0,2.0,Walt Disney Pictures


In [18]:
#creo le associazioni tra i i e le sigle(chiavi) dei paesi di produzione
countries_id = pd.DataFrame()
for i in range(len(df['production_countries'])):
    unrolled_data = pd.read_json(df['production_countries'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    countries_id = pd.concat([countries_id, unrolled_data])
countries_id = countries_id.rename(index=str, columns={'iso_3166_1': 'country_id'})
countries_link = countries_id[['country_id', 'name']]
countries_id = countries_id[['movie_id', 'country_id']].set_index(['movie_id', 'country_id'])
countries_id.head()

Empty DataFrame
Columns: []
Index: [(19995, US), (19995, GB), (285, US), (206647, GB), (206647, US)]

In [19]:
# Creo la tabella delle associazioni
countries_link = countries_link.set_index(['country_id'])
countries_link = countries_link.drop_duplicates()
countries_link.head()

,name
country_id,
US,United States of America
GB,United Kingdom
JM,Jamaica
BS,Bahamas
DM,Dominica


In [20]:
#id dei i con le sigle dei linguaggi
languages_id = pd.DataFrame()
for i in range(len(df['spoken_languages'])):
    unrolled_data = pd.read_json(df['spoken_languages'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    languages_id = pd.concat([languages_id, unrolled_data])
languages_id = languages_id.rename(index=str, columns={'iso_639_1': 'language_id'})
languages_link = languages_id[['language_id', 'name']]
languages_id = languages_id[['movie_id', 'language_id']].set_index(['movie_id', 'language_id'])
languages_id.head()

Empty DataFrame
Columns: []
Index: [(19995, en), (19995, es), (285, en), (206647, fr), (206647, en)]

In [21]:
# id dei linguaggi e il loro corrispettivo
languages_link = languages_link.set_index(['language_id'])
languages_link = languages_link.drop_duplicates()
languages_link.head()

,name
language_id,
en,English
es,Español
fr,Français
it,Italiano
de,Deutsch


In [22]:
#id dei credit 
cast = pd.DataFrame()
for i in range(len(df['cast'])):
    unrolled_data = pd.read_json(df['cast'][i])
    temp_id = df['movie_id'][i]
    unrolled_data['movie_id'] = temp_id
    cast = pd.concat([cast, unrolled_data])
cast = cast.rename(index=str, columns={'id': 'person_id'})
#cast_id = cast[['movie_id', 'credit_id']]
cast_id = cast[['movie_id', 'cast_id', 'person_id', 'credit_id', 'character','order']]
cast_id = cast_id.set_index(['credit_id'])
cast_id.head()

,movie_id,cast_id,person_id,character,order
credit_id,,,,,
5602a8a7c3a3685532001c9a,19995,242.0,65731.0,Jake Sully,0.0
52fe48009251416c750ac9cb,19995,3.0,8691.0,Neytiri,1.0
52fe48009251416c750aca39,19995,25.0,10205.0,Dr. Grace Augustine,2.0
52fe48009251416c750ac9cf,19995,4.0,32747.0,Col. Quaritch,3.0
52fe48009251416c750ac9d3,19995,5.0,17647.0,Trudy Chacon,4.0


In [23]:
#id crew
crew = pd.DataFrame()
for i in range(len(df['crew'])):
    unrolled_data = pd.read_json(df['crew'][i])
    temp_id = df['movie_id'][i]
    unrolled_data['movie_id'] = temp_id
    crew = pd.concat([crew, unrolled_data])
crew = crew.rename(index=str, columns={'id': 'person_id'})
#crew_id = crew[['movie_id', 'credit_id']]
crew_id = crew[['movie_id', 'credit_id', 'person_id', 'job', 'department']]
crew_id = crew_id.set_index(['credit_id'])
crew_id.head()

,movie_id,person_id,job,department
credit_id,,,,
52fe48009251416c750aca23,19995,1721.0,Editor,Editing
539c47ecc3a36810e3001f87,19995,496.0,Production Design,Art
54491c89c3a3680fb4001cf7,19995,900.0,Sound Designer,Sound
54491cb70e0a267480001bd0,19995,900.0,Supervising Sound Editor,Sound
539c4a4cc3a36810c9002101,19995,1262.0,Casting,Production


In [24]:
#unisco crew e cast
people_crew = crew[['person_id', 'gender', 'name']]
people_cast = cast[['person_id', 'gender', 'name']]
people = pd.concat([people_crew, people_cast])
people = people.sort_values('person_id').drop_duplicates().set_index('person_id')
people.head()

,gender,name
person_id,,
1.0,2.0,George Lucas
2.0,2.0,Mark Hamill
3.0,2.0,Harrison Ford
4.0,1.0,Carrie Fisher
5.0,2.0,Peter Cushing


6 For each movie, compute the gross margin (difference between revenue and budget)

In [25]:
df["gross_margin"]=df["revenue"]-df["budget"]
df[["movie_id","gross_margin"]].head(10)

,movie_id,gross_margin
0,19995,2550965087
1,285,661000000
2,206647,635674609
3,49026,834939099
4,49529,24139100
5,559,632871626
6,38757,331794936
7,99861,1125403694
8,767,683959197
9,209112,623260194


7 For each movie, compute the number of crew members

In [26]:
#trovo numero di membri della crew contando il numero di sottodizionari nel file JSON "crew"
df["no. crew members"]=df.crew.apply(lambda x: len(json.loads(x)))
df[["movie_id","no. crew members"]].head(10)

,movie_id,no. crew members
0,19995,153
1,285,32
2,206647,155
3,49026,217
4,49529,132
5,559,56
6,38757,79
7,99861,74
8,767,27
9,209112,111


8 For each movie, compute the number of directors

In [27]:
#trovo numero di directors controllando la lunghezza del vettore dir dove compaiono tutti i Director del film 
df['crew']=df.crew.apply(json.loads)
df["no. directors"]=df.crew.apply(lambda crew_members: len([dir for dir in crew_members if dir["job"]=="Director"]))
df[["no. directors","title"]].groupby("no. directors").count()

,title
no. directors,
0,30
1,4465
2,277
3,21
4,3
6,2
7,1
8,1
12,2


9 For each language, compute the number of movies where such language is spoken.

In [28]:
#no. movies for spoken_language
lang_count=languages_id.reset_index(level=0).groupby('language_id').count()
film4language=pd.merge(lang_count, languages_link, left_index=True, right_index=True, how = 'inner')
film4language.set_index('name',inplace=True)

#escludo missing values per le lingue
film4language.drop(["??????",""],axis=0,inplace=True)
film4language.head(7)

,movie_id
name,
Afrikaans,7
العربية,67
български език,3
Bamanankan,1
বাংলা,1
Bosanski,2
Català,1


10 For each company and each decade, compute the overall revenue

In [29]:
#revenues for production_company

df['revenue/company']=df['revenue']/df.production_companies.apply(lambda x:len(json.loads(x)) if len(json.loads(x))!=0 else 1 )
revenue4filmId=pd.merge(production_id.reset_index(),df[['revenue/company','movie_id']],on='movie_id',how='outer')
revenue_count=revenue4filmId.groupby('company_id').sum()
revenue_comp=pd.merge(revenue_count, production_link.reset_index(level=0).set_index('company_id'), left_index=True, right_index=True, how = 'left')
revenue4company=revenue_comp.drop_duplicates('name').set_index('name').drop(['movie_id','index'],axis=1)
pd.options.display.float_format = '{:,.2f}'.format  # fixing scientific notation display
revenue4company.sort_values('revenue/company',ascending=False).rename(columns={revenue4company.columns[0]:"Revenue($)"}).head(7)

,Revenue($)
name,
Paramount Pictures,"15,752,336,903.39"
Warner Bros.,"15,091,181,359.17"
Universal Pictures,"14,754,247,538.17"
Walt Disney Pictures,"12,800,096,261.40"
Twentieth Century Fox Film Corporation,"12,603,027,682.93"
Columbia Pictures,"10,863,224,555.77"
DreamWorks Animation,"8,417,098,052.25"


In [30]:
#revenues for decades
df["decade"]=df.year.str[:3]+"0"
rev_per_decade=df[['revenue','decade']].groupby('decade').sum()
#drop del dato con anno mancante
#rev_per_decade.drop("nan0",axis=0,inplace=True)

rev_per_decade.rename(columns={rev_per_decade.columns[0]:"Revenue($)"}).head(11)

,Revenue($)
decade,
1910,8394751
1920,27008422
1930,671309038
1940,592750774
1950,383942165
1960,2228628931
1970,6543658156
1980,16963368503
1990,61301228838


11 For each decade, compute the company with maximum revenue

In [31]:
temp=pd.merge(production_id.reset_index(),df[['revenue/company','movie_id','decade']],on='movie_id',how='outer')
temp=temp[['revenue/company','company_id','decade']].dropna().set_index(['company_id', 'decade']).reset_index().sort_values(by='decade')
revenue_comp_decade=pd.merge(temp, production_link, on='company_id', how = 'left').set_index('company_id')
revenue_comp_decade.pivot_table(index='name',columns=['decade'],aggfunc=sum).idxmax()

                 decade
revenue/company  1910                   Triangle Film Corporation
                 1920                   Metro-Goldwyn-Mayer (MGM)
                 1930                   Metro-Goldwyn-Mayer (MGM)
                 1940                     Walt Disney Productions
                 1950      Twentieth Century Fox Film Corporation
                 1960                             Eon Productions
                 1970                          Universal Pictures
                 1980                          Paramount Pictures
                 1990                          Paramount Pictures
                 2000                                Warner Bros.
                 2010                              Marvel Studios
dtype: object

12 In each year, how many movies have revenue smaller than the budget?

In [32]:
fail_per_year=df[df["revenue"]<df["budget"]][['id','year']].groupby('year').count()
fail_per_year.columns=['Failures']
fail_per_year.head(10)

,Failures
year,
1927,1
1947,1
1948,1
1954,1
1962,1
1963,1
1964,2
1965,3
1966,1


### Punti da due persone

1 Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

La formula utilizzata deriva dall'osservazione che la serie si sviluppa secondo la seguente formula:
$$\sum_{i=0}^{n-1} \frac{2^{n-i-1}}{2^{n}-1}$$


In [33]:
df["cast"]=df.cast.apply(json.loads)
df['actors']=df.cast.apply(lambda x: [act['name'] for act in x])
#df['actors']=df.actors.apply(lambda x: [re.sub('^ +','',x[i]) for i in range(len(x))])
#df['actors']=df.actors.apply(lambda x: list(map(lambda y: re.sub('^ +','',y),x)))

In [34]:
revenue=pd.DataFrame() 
act_name=[]
act_rev=[]
mov_name=[]
for index,row in df.iterrows():
    n=len(row["actors"])
    for i in range(n):
        act_name.append(row['actors'][i])
        act_rev.append((2**(n-i-1)/(2**n-1))*row["revenue"])    
        mov_name.append(row["original_title"])
revenue['movie']=mov_name
revenue['actor'] =act_name
revenue['revenue']=act_rev

In [35]:
#checking that the revenue for interstellar is right (675120017)
round(sum(revenue[revenue['movie']=='Interstellar']["revenue"]))

675120017.0

In [36]:
revenue.head(15)

,movie,actor,revenue
0,Avatar,Sam Worthington,"1,393,982,543.50"
1,Avatar,Zoe Saldana,"696,991,271.75"
2,Avatar,Sigourney Weaver,"348,495,635.88"
3,Avatar,Stephen Lang,"174,247,817.94"
4,Avatar,Michelle Rodriguez,"87,123,908.97"
5,Avatar,Giovanni Ribisi,"43,561,954.48"
6,Avatar,Joel David Moore,"21,780,977.24"
7,Avatar,CCH Pounder,"10,890,488.62"
8,Avatar,Wes Studi,"5,445,244.31"
9,Avatar,Laz Alonso,"2,722,622.16"


2 For each actor find the total revenue attributed to him/her.

In [37]:
#overall revenue for actor
actors_revenue=revenue.groupby('actor').sum()
#showing random split
num_actors=len(actors_revenue)
start=random.randrange(1,num_actors)
actors_revenue.iloc[start:start+10]

,revenue
actor,
Carryl Lynn,0.00
Carsey Walker Jr.,355.84
Carson Aune,1.13
Carson D. Mell,360.60
Carson Flowers,91.36
Carson Grant,0.21
Carson Kressley,0.00
Carson Manning,0.00
Carson Nattrass,51.87


3 Find the actor that is responsible for the most overall revenue.

In [38]:
#top 3 most payed actors
actors_revenue.sort_values('revenue',ascending=False).head(3)

,revenue
actor,
Tom Cruise,"3,976,119,762.06"
Tom Hanks,"3,800,124,342.96"
Robert Downey Jr.,"3,631,739,495.78"
